In [6]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split, DataLoader
from PIL import Image
import pandas as pd
import os

# 检查CUDA设备是否可用，选择设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# 数据路径
train_dir = "F:\\python代码\\机器学习\\期末作业\\train"
pred_dir = "F:\\python代码\\机器学习\\期末作业\\pred"
csv_dir = "F:\\python代码\\机器学习\\期末作业\\pre_data.csv"
# 标签字典
label_dict = {0: 'cavallo', 1: 'mucca', 2: 'scoiattolo'}



Using device: cuda:0


In [7]:

# 数据预处理
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# 加载数据集
train_data = datasets.ImageFolder(train_dir, transform=transform)
# 划分数据集
train_size = int(0.7 * len(train_data))
val_size = int(0.15 * len(train_data))
test_size = len(train_data) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(train_data, [train_size, val_size, test_size])

# 数据加载器
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# 构建模型并转移到GPU
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)
model = model.to(device)  # 转移到GPU

f:\python\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
f:\python\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 训练模型
num_epochs = 5  # 根据需要调整
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # 数据也要转移到GPU
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")



Epoch 1/5, Loss: 0.5028612159352538
Epoch 2/5, Loss: 0.2914784305835452
Epoch 3/5, Loss: 0.18517327967350253
Epoch 4/5, Loss: 0.1587056686191638
Epoch 5/5, Loss: 0.154168549028347


In [9]:

# 验证模型
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # 数据也要转移到GPU
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Validation Accuracy: {100 * correct // total} %')


Validation Accuracy: 95 %


In [10]:

# 测试模型
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # 数据也要转移到GPU
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Accuracy: {100 * correct // total} %')


Test Accuracy: 96 %


In [12]:
pred_dir = "F:\\python代码\\机器学习\\期末作业\\pred"   #输入待预测数据集地址
csv_dir = "F:\\python代码\\机器学习\\期末作业\\pre_data.csv" #输出文件

# 预测未标记数据
pred_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

pred_images = os.listdir(pred_dir)
predictions = []

for image_name in pred_images:
    image_path = os.path.join(pred_dir, image_name)
    image = Image.open(image_path).convert('RGB')
    image = pred_transform(image).unsqueeze(0)
    image = image.to(device)  # 数据转移到GPU

    output = model(image)
    _, predicted = torch.max(output, 1)
    predictions.append(label_dict[predicted.item()])  # 使用类别名

# 保存预测结果
pred_labels = ['image_' + str(i+1) + '.jpeg' for i in range(len(predictions))]
pred_data = list(zip(pred_labels, predictions))
pred_df = pd.DataFrame(pred_data, columns=["Image Name", "Label"])
pred_df.to_csv(csv_dir, index=False)
print("预测完成，结果已保存。")

预测完成，结果已保存。
